In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from underthesea import word_tokenize, pos_tag, sent_tokenize
import warnings
from gensim import corpora, models, similarities
import re

In [3]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split

In [4]:
STOP_WORD_FILE = 'vietnamese-stopwords.txt'

In [5]:
with open(STOP_WORD_FILE, 'r', encoding='utf-8') as file:
    stop_words = file.read()

stop_words = stop_words.split('\n')

In [6]:
# 1. Load dữ liệu
df_1 = pd.read_csv('Products_ThoiTrangNam_raw.csv')
df_2 = pd.read_csv('Products_ThoiTrangNam_rating_raw.csv')

In [7]:
# Fill NA cho 'description' và 'image'
df_1['description'] = df_1['description'].fillna('')
df_1['image'] = df_1['image'].fillna('')

In [8]:
# Hàm kiểm tra từ có phải là từ tiếng Việt "sạch"
def is_valid_vietnamese(word):
    vietnamese_chars = (
        "a-zA-Z0-9_"
        "àáạảãâầấậẩẫăằắặẳẵ"
        "èéẹẻẽêềếệểễ"
        "ìíịỉĩ"
        "òóọỏõôồốộổỗơờớợởỡ"
        "ùúụủũưừứựửữ"
        "ỳýỵỷỹ"
        "đ"
        "ÀÁẠẢÃÂẦẤẬẨẪĂẰẮẶẲẴ"
        "ÈÉẸẺẼÊỀẾỆỂỄ"
        "ÌÍỊỈĨ"
        "ÒÓỌỎÕÔỒỐỘỔỖƠỜỚỢỞỠ"
        "ÙÚỤỦŨƯỪỨỰỬỮ"
        "ỲÝỴỶỸ"
        "Đ"
    )
    pattern = f'^[{vietnamese_chars}]+$'
    return re.match(pattern, word) is not None

In [9]:
# Hàm xử lý một mô tả
def filter_vietnamese_words(text):
    if not isinstance(text, str):
        return ''
    words = text.split()
    clean_words = [w for w in words if is_valid_vietnamese(w)]
    return ' '.join(clean_words)

In [10]:
# Áp dụng lọc
df_1['description_clean'] = df_1['description'].apply(filter_vietnamese_words)


print(df_1[['description', 'description_clean']])

                                             description  \
0      Danh Mục\nShopee\nThời Trang Nam\nÁo Ba Lỗ\nCh...   
1      Danh Mục\nShopee\nThời Trang Nam\nÁo Ba Lỗ\nXu...   
2      Danh Mục\nShopee\nThời Trang Nam\nÁo Ba Lỗ\nTh...   
3      Danh Mục\nShopee\nThời Trang Nam\nÁo Ba Lỗ\nCh...   
4      Danh Mục\nShopee\nThời Trang Nam\nÁo Ba Lỗ\nCh...   
...                                                  ...   
49658  Danh Mục\nShopee\nThời Trang Nam\nVớ/Tất\nChiề...   
49659  Danh Mục\nShopee\nThời Trang Nam\nVớ/Tất\nThươ...   
49660  Danh Mục\nShopee\nThời Trang Nam\nVớ/Tất\nChất...   
49661  Danh Mục\nShopee\nThời Trang Nam\nVớ/Tất\nKho ...   
49662  Danh Mục\nShopee\nThời Trang Nam\nVớ/Tất\nXuất...   

                                       description_clean  
0      Danh Mục Shopee Thời Trang Nam Áo Ba Lỗ Chiều ...  
1      Danh Mục Shopee Thời Trang Nam Áo Ba Lỗ Xuất x...  
2      Danh Mục Shopee Thời Trang Nam Áo Ba Lỗ Thương...  
3      Danh Mục Shopee Thời Trang Nam Áo Ba

In [11]:
# Bỏ phần 'Danh Mục Shopee Thời Trang Nam ' ở đầu mỗi dòng
df_1['description_clean'] = df_1['description_clean'].str.replace('Danh Mục Shopee Thời Trang Nam ', '', regex=False)

In [12]:
# Chọn cột và đổi tên luôn
df_1 = df_1[['product_id', 'product_name', 'description_clean']].rename(columns={
    'product_id': 'ma_san_pham',
    'product_name': 'ten_san_pham',
    'description_clean': 'mo_ta'
})

print(df_1.head(5))

   ma_san_pham                                       ten_san_pham  \
0          190              Áo ba lỗ thun gân ,form body tôn dáng   
1          191  Áo Ba Lỗ Nam Trắng Chất Cotton Siêu Mát, Siêu Đẹp   
2          192  Áo Ba Lỗ Nam Tyasuo chất vải co dãn mát, không...   
3          193                 ÁO BA LỖ HÀNG VIỆT NAM 100% COTTON   
4          194  Áo Thun Nam Thể Thao Ba Lỗ Mẫu Mới Siêu Đẹp (B...   

                                               mo_ta  
0  Áo Ba Lỗ Chiều dài tay áo Khác Phong cách Thể ...  
1  Áo Ba Lỗ Xuất xứ Việt Nam Tên tổ chức chịu trá...  
2  Áo Ba Lỗ Thương hiệu Tyasuo Chiều dài tay áo K...  
3  Áo Ba Lỗ Chất liệu Cotton Kho hàng 7981 Gửi từ...  
4  Áo Ba Lỗ Chiều dài tay áo Không tay Phong cách...  


In [13]:
df_1= df_1.set_index('ma_san_pham')

# Lưu ra file CSV
df_1.to_csv('San_pham_temp.csv', index=True)

In [14]:
df_1['Content'] = df_1['mo_ta'].apply(lambda x: ' '.join(x.split()[:200]))

In [15]:
df_1.head()

,ten_san_pham,mo_ta,Content
ma_san_pham,,,
190,"Áo ba lỗ thun gân ,form body tôn dáng",Áo Ba Lỗ Chiều dài tay áo Khác Phong cách Thể ...,Áo Ba Lỗ Chiều dài tay áo Khác Phong cách Thể ...
191,"Áo Ba Lỗ Nam Trắng Chất Cotton Siêu Mát, Siêu Đẹp",Áo Ba Lỗ Xuất xứ Việt Nam Tên tổ chức chịu trá...,Áo Ba Lỗ Xuất xứ Việt Nam Tên tổ chức chịu trá...
192,"Áo Ba Lỗ Nam Tyasuo chất vải co dãn mát, không...",Áo Ba Lỗ Thương hiệu Tyasuo Chiều dài tay áo K...,Áo Ba Lỗ Thương hiệu Tyasuo Chiều dài tay áo K...
193,ÁO BA LỖ HÀNG VIỆT NAM 100% COTTON,Áo Ba Lỗ Chất liệu Cotton Kho hàng 7981 Gửi từ...,Áo Ba Lỗ Chất liệu Cotton Kho hàng 7981 Gửi từ...
194,Áo Thun Nam Thể Thao Ba Lỗ Mẫu Mới Siêu Đẹp (B...,Áo Ba Lỗ Chiều dài tay áo Không tay Phong cách...,Áo Ba Lỗ Chiều dài tay áo Không tay Phong cách...


In [16]:
# word_tokenize
df_1["Content_wt"]=df_1["Content"].apply(lambda x: word_tokenize(x, format="text"))

In [18]:
df_1[["Content", "Content_wt"]].head(3)

,Content,Content_wt
ma_san_pham,,
190,Áo Ba Lỗ Chiều dài tay áo Khác Phong cách Thể ...,Áo Ba_Lỗ Chiều dài tay_áo Khác Phong_cách Thể_...
191,Áo Ba Lỗ Xuất xứ Việt Nam Tên tổ chức chịu trá...,Áo Ba_Lỗ Xuất_xứ Việt_Nam Tên tổ_chức chịu trá...
192,Áo Ba Lỗ Thương hiệu Tyasuo Chiều dài tay áo K...,Áo Ba_Lỗ Thương_hiệu Tyasuo Chiều dài tay_áo K...


In [27]:
# Vector hóa nội dung
vectorizer = TfidfVectorizer(analyzer='word', stop_words=stop_words,max_features=1000,dtype=np.float32)
tfidf_matrix = vectorizer.fit_transform(df_1['Content_wt'])

In [35]:
from sklearn.neighbors import NearestNeighbors

nn_model = NearestNeighbors(
    n_neighbors=6,              # 5 gợi ý + chính nó
    metric="cosine",
    algorithm="brute"
)
nn_model.fit(tfidf_matrix)


NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=6)

In [40]:
indices = pd.Series(df_1.index, index=df_1.index)  # identity mapping vì đã là index

In [44]:
def get_recommendations_nn(ma_san_pham, top_n=3):
    try:
        idx = df_1.index.get_loc(ma_san_pham)
    except KeyError:
        return f"❌ Không tìm thấy mã sản phẩm: {ma_san_pham}"

    distances, indices_found = nn_model.kneighbors(tfidf_matrix[idx], n_neighbors=top_n + 1)
    result_indices = indices_found[0][1:]  # Bỏ chính nó
    return df_1.iloc[result_indices][['ten_san_pham','Content']]

In [45]:
get_recommendations_nn(190, top_n=3)

,ten_san_pham,Content
ma_san_pham,,
181072,Men's printed short sleeve men's floral shirt T32,Áo Áo Sơ Mi Kiểu cổ áo Cổ áo truyền thống Chất...
18393,"Áo thun Big Size GẤU 194 vải coton mềm mịn, co...",Áo Áo Thun Thương hiệu Gấu Phong cách Hàn Quốc...
20309,Áo hoodie kiểu dáng đơn trẻ trung sành điệu ch...,Áo Áo Len Áo Nỉ Áo Hoodie Mẫu In Kho hàng 5880...


In [47]:
import pickle

# 1. Lưu model KNN
with open('nn_model.pkl', 'wb') as f:
    pickle.dump(nn_model, f)

# 2. Lưu tfidf_matrix
with open('tfidf_matrix.pkl', 'wb') as f:
    pickle.dump(tfidf_matrix, f)

# 3. Lưu dataframe sản phẩm
with open('df_products.pkl', 'wb') as f:
    pickle.dump(df_1, f)

print("✅ Đã lưu xong tất cả các file!")


✅ Đã lưu xong tất cả các file!


In [48]:
# 1. Load lại các object
with open('nn_model.pkl', 'rb') as f:
    nn_model = pickle.load(f)

with open('tfidf_matrix.pkl', 'rb') as f:
    tfidf_matrix = pickle.load(f)

with open('df_products.pkl', 'rb') as f:
    df_1 = pickle.load(f)

# 2. Viết lại hàm get_recommendations_nn
def get_recommendations_nn(ma_san_pham, top_n=3):
    try:
        idx = df_1.index.get_loc(ma_san_pham)
    except KeyError:
        return f"❌ Không tìm thấy mã sản phẩm: {ma_san_pham}"

    distances, indices_found = nn_model.kneighbors(tfidf_matrix[idx], n_neighbors=top_n + 1)
    result_indices = indices_found[0][1:]  # Bỏ chính nó
    return df_1.iloc[result_indices][['ten_san_pham', 'Content']]


In [49]:
get_recommendations_nn(190, top_n=3)

,ten_san_pham,Content
ma_san_pham,,
181072,Men's printed short sleeve men's floral shirt T32,Áo Áo Sơ Mi Kiểu cổ áo Cổ áo truyền thống Chất...
18393,"Áo thun Big Size GẤU 194 vải coton mềm mịn, co...",Áo Áo Thun Thương hiệu Gấu Phong cách Hàn Quốc...
20309,Áo hoodie kiểu dáng đơn trẻ trung sành điệu ch...,Áo Áo Len Áo Nỉ Áo Hoodie Mẫu In Kho hàng 5880...
